# Safe Interval Path Planning On a Grid :: SIPP Algorithm (based on A* with idea of safe intervals)

In this project, we will consider the problem of finding the fastest safe path on a graph with a special structure, widely used in robotics and computer games — a grid. The grid consists of both free and blocked cells, and an agent can move from one free cell to another. Our grid will also have some obstacles and they will be moving. Algortihm needs to find a route that does not intrfere with obstacles at any point in our discrete time




In [1]:

from typing import Callable, Iterable, Optional, Tuple, Type

from node import Node
from sipp_map import Map
from search_tree import SearchTreePQD


In [2]:
test_name = "lak109d_75_0"

In [3]:
task_path = f"../data/{test_name}.xml"
log_path = f"../logs/log_{test_name}.xml"

In [4]:
test_map = Map(task_path)

/opt/homebrew/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


## SIPP ≈ A* Algorithm Without Reexpansions

The primary component of the `A*` algorithm is the heuristic function.

Ideally, this function should be admissible (it never overestimates the true cost to the goal) and consistent (satisfies the triangle inequality). The Manhattan distance serves as an admissible and consistent heuristic for 4-connected grids. Therefore, if we integrate it into the `Dijkstra` search loop from the previous lab, we'll obtain the `A*` algorithm.

Let's proceed!


The input for the `A*` algorithm implementation is the same as for `Dijkstra`:

The input is:
- map representation
- start/goal cells
- heuristic function $^*$
- a reference to the implementation of the SearchTree

The output is:
- path found flag (`true` or `false`)
- last node of the path (so one can unwind it using the parent-pointers and get the full path)
- the number of steps (iterations of the main loop)
- the number of nodes that compose the search tree at the final iteration of the algorithm (=the size of the resultant search tree)
- OPEN and CLOSED (as iterable collections of nodes) for further visualization purposes


PS: You might also want to display, at the final iteration, the number of OPEN duplicates encountered during the search, as shown below:

```print("During the search, the following number of OPEN dublicates was encountered: ", ast.number_of_open_duplicates) ```


In [5]:
def sipp(
    task_map: Map,
    start_i: int,
    start_j: int,
    goal_i: int,
    goal_j: int,
    heuristic_func: Callable,
    search_tree: Type[SearchTreePQD],
) -> Tuple[bool, Optional[Node], int, int, Optional[Iterable[Node]], Optional[Iterable[Node]]]:
    """
    Implements the A* search algorithm.

    Parameters
    ----------
    task_map : Map
        The grid or map being searched.
    start_i, start_j : int, int
        Starting coordinates.
    goal_i, goal_j : int, int
        Goal coordinates.
    heuristic_func : Callable
        Heuristic function for estimating the distance from a node to the goal.
    search_tree : Type[SearchTreePQD]
        The search tree to use.

    Returns
    -------
    Tuple[bool, Optional[Node], int, int, Optional[Iterable[Node]], Optional[Iterable[Node]]]
        Tuple containing:
        - A boolean indicating if a path was found.
        - The last node in the found path or None.
        - Number of algorithm iterations.
        - Size of the resultant search tree.
        - OPEN set nodes for visualization or None.
        - CLOSED set nodes.
    """
    ast = search_tree()
    steps = 0

    start_node = Node(start_i, start_j, interval=0, g=0, h=heuristic_func(start_i, start_j, goal_i, goal_j))
    task_map.set_intervals(start_i, start_j)
    ast.add_to_open(start_node)

    while not ast.open_is_empty():
        current = ast.get_best_node_from_open()

        steps += 1

        if current is None:
            break

        if current.i == goal_i and current.j == goal_j:  # is goal
            return True, current, steps, len(ast), ast.opened, ast.expanded
        
        ast.add_to_closed(current)


        for i, j, cost1, cost2, interval in task_map.get_successors(current):
            new = Node(i, j, interval)
            if not ast.was_expanded(new):

                new.parent = current

                new.h = heuristic_func(i, j, goal_i, goal_j)
                new.g = max(current.g + cost1, cost2)
                new.f = new.g + new.h

                ast.add_to_open(new)

    return False, None, steps, len(ast), None, ast.expanded

In [6]:
_, current, steps, pathlen, opened, _ = sipp(test_map, test_map.start_j, test_map.start_i, test_map.goal_j, test_map.goal_i, test_map.get_distance, SearchTreePQD)

## Восстановим путь

In [7]:

lst = [current]
while current.parent is not None:
    lst.append(current.parent)
    current = current.parent

In [8]:
lst = list(reversed(lst))

points = []

In [9]:
for index, point in enumerate(lst):
    
    if index == 0:
        points.append(point)
        continue

    if point.g - lst[index-1].g != 1:
        time = lst[index-1].g + 1
        while time < point.g:
            points.append(Node(lst[index-1].i, lst[index-1].j, time, g=time))
            time += 1
    
    points.append(point)


## Visualisation

In [10]:
with open(task_path, 'r') as f:
    task = f.readlines()


In [11]:
with open(log_path, 'w') as f:
    print(''.join(task[:-1]), file=f)

    print("    <log>", file=f)
    print(f"        <summary pathlength=\"{len(points)}\" numberofsteps=\"{steps}\" nodescreated=\"{len(opened)}\" searchtime=\"1\"/>", file=f)
    print("        <path>", file=f)

    for el in points:
        print(f"            <point x=\"{el.j + 1}\" y=\"{el.i + 1}\" time=\"{el.g}\"/>", file=f)

    print("         </path>", file=f)
    print("    </log>", file=f)
    print(task[-1], file=f)

To get video visualization ```python3 src/visualization/visualize.py logs/log_simple.xml -o outputs/output_file_log.mp4```